In [ ]:
import yaml
import glob
import os 
import itertools
import re
from collections import OrderedDict
import pickle
import multiprocessing
from collections.abc import MutableMapping
from IPython import display as ICD
from decimal import Decimal
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch

from paper_utils.plot_utils import *

In [ ]:
plotly_dir = '../plotly'
datasets = ['CoraML', 'CiteSeer', 'PubMed', 'AmazonPhotos', 'AmazonComputers', 'CoauthorCS', 'CoauthorPhysics']
param_keys = ['dist_reg', 'orig_dist_reg', 'KNN_K', 'dist_sigma', 'dist_embedding_beta', 'activation_type', 'entropy_reg', 'lipschitz_reg', 'lipschitz_init', 'num_layers']
dataset_keys = ['dataset', ]
# result_keys = ['val_accuracy', 'test_ood_detection_epistemic_auroc', 'test_accuracy', 'confidence_epistemic_auroc']

## Convert ray_result files to a compiled csv

In [ ]:
directories = os.listdir("../ray_results/")
for a_dir in directories:
    if ".~lock" in a_dir:
        continue
    if not a_dir.endswith(".csv") and "hidden" not in a_dir:
        file_to_write = f'../ray_results/{a_dir}.csv'
        if not os.path.exists(file_to_write):
            print(file_to_write + " is writing!")
            df = get_df(a_dir)
            df.to_csv(f'../ray_results/{a_dir}.csv')
            print('Finished')
        else:
            pass
            print(file_to_write + " already exists!") 

In [ ]:
trials = sorted(glob.glob("../ray_results/*.csv"))
results = []
full_results = []
   
outputs_config = [
    'config.model.activation_type', 'config.model.entropy_reg', 'config.model.dist_reg', 'config.model.orig_dist_reg', 'config.model.decoder_reg'
]
outputs_loc = [
            'val_CE', 'test_accuracy', 'test_ood_detection_aleatoric_auroc', 'test_ood_detection_epistemic_auroc', 'test_ood_detection_features_auroc', 
            'test_ood_detection_aleatoric_apr', 'test_ood_detection_epistemic_apr', 'test_ood_detection_features_apr']
outputs_miss = [
    'val_CE', 'test_accuracy', 'test_confidence_aleatoric_auroc', 'test_confidence_epistemic_auroc', 'test_confidence_aleatoric_apr', 'test_confidence_epistemic_apr'
]
out = outputs_config + outputs_loc

avoid_words = ['sqrt', 'final', 'isolated', 'budget', 'dice', 'preserving', 'lipschitz', 'stress', 'leave', 'best']
# avoid_words = []
big_full = []

for a_trial in trials:
    filename = os.path.basename(a_trial)
    name, _ = os.path.splitext(filename)
    df = pd.read_csv(a_trial)
    df = df.drop(columns="Unnamed: 0")

    if 'decoder' not in name.casefold():
        continue
    print(name)
    
    if all([a_avoidable not in name.casefold() for a_avoidable in avoid_words]):
        try: 
            result, full_result = get_best_result(df, 'config.data.dataset', 'val_CE', outputs_config + outputs_loc)
        except:
            result, full_result = get_best_result(df, 'config.data.dataset', 'val_CE', outputs_config + outputs_miss)
            
        if len(result) == 0:
            continue

        result['name'] = name
        full_result['name'] = name

        results.append(result)
        full_results.append(full_result)
        big_full.append(df)
    else:
        continue

    # for y in ['accuracy', ]:#, 'test_ood_detection_epistemic_auroc', 'test_confidence_aleatoric_auroc', 'test_confidence_aleatoric_apr']:
    #     print(name)
    #     plot_data(plotly_dir, df, name, y, 'tune-reg', as_line=True, log_x=True, 
    #                 facet_x='config.data.dataset', facet_y='config.model.activation_type', line_dash=None)

    # if "Isolated" in a_trial or "Leave_Out_Classes" in a_trial:
    #     df = generate_latex_table(df, ['config.data.dataset'], ['test_ood_detection_aleatoric_auroc', 'test_ood_detection_epistemic_auroc', 'test_accuracy'])
    #     ICD.display(df)
    
results = pd.concat(results)
full_results = pd.concat(full_results)
results.columns = map(lambda s:s.replace("Confidence", "").strip(), results.columns)
pd.set_option('display.max_rows', results.shape[0]+1) #  'w/o Net AUROC' 'w/o Net APR'
for a_key in ['Accuracy', 'Alea AUROC', 'Epist AUROC', 'Alea APR', 'Epist APR']:
    results[a_key] = results[a_key].apply(lambda x: np.round(x*100, 2))

In [ ]:
results

In [ ]:
def get_partial_key(a_partial_key):
    keys = df.columns
    return [a_key for a_key in keys if a_partial_key in a_key]
print(get_partial_key('val_CE'))

In [ ]:
big_full = pd.concat(big_full)
big_full = big_full[['PubMed' in a_name for a_name in big_full['config.data.dataset']]]
big_full

In [ ]:
# results = results.sort_values(['Dataset', "CE"])
# dfs = [x for _, x in results.groupby('Dataset')]
# all_dfs = []
# for a_df in dfs:
#     # graph_row = a_df[['Graph' in a_name for a_name in a_df.name]].iloc[0]
#     # entropy_row = a_df[['Entropy' in a_name for a_name in a_df.name]].iloc[0]
#     entropy_row = a_df[['Decoder' in a_name for a_name in a_df.name]].iloc[0]
#     df = pd.DataFrame([big_full, entropy_row], columns=big_full.keys())
#     all_dfs.append(df)
    
# ICD.display(pd.concat(all_dfs))